In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

## 2D gene expression profiles loading (X)



In [2]:
with open('X_brca.pkl', 'rb') as f:
    X_brca = pickle.load(f)

In [3]:
with open('X_non_brca.pkl', 'rb') as f:
    X_non_brca = pickle.load(f)

In [4]:
print('There are {} brca and {} non-brca gene-expression-profiles'.format(len(X_brca), len(X_non_brca)))

There are 1196 brca and 9224 non-brca gene-expression-profiles


## Visualizing some images

## Survival data loading (Y)

In [5]:
brca_clinical = pd.read_hdf('../data/TCGA_data.h5', key='brca_clinical').sort_index()
non_brca_clinical = pd.read_hdf('../data/TCGA_data.h5', key='non_brca_clinical').sort_index()

In [6]:
len(brca_clinical)+len(non_brca_clinical)

10497

In [7]:
# Deleting samples with 'NaN's in OS-columns 

## non_brca
non_brca_clinical['OS.time'] = non_brca_clinical['OS.time'].map(lambda x: np.nan if x == 'NaN' else x)
non_brca_clinical['OS'] = non_brca_clinical['OS'].map(lambda x: np.nan if x == 'NaN' else x)
non_brca_clinical.dropna(subset=['OS.time', 'OS'], inplace=True)
non_brca_clinical['OS.time'] = non_brca_clinical['OS.time'].astype(float)
non_brca_clinical['OS'] = non_brca_clinical['OS'].astype(float)

## brca
brca_clinical['OS.time'] = brca_clinical['OS.time'].map(lambda x: np.nan if x == 'NaN' else x)
brca_clinical['OS'] = brca_clinical['OS'].map(lambda x: np.nan if x == 'NaN' else x)
brca_clinical.dropna(subset=['OS.time', 'OS'], inplace=True)
brca_clinical['OS.time'] = brca_clinical['OS.time'].astype(float)
brca_clinical['OS'] = brca_clinical['OS'].astype(float)

In [8]:
print('There are {} brca clinical info and {} non-brca clinical info'.format(len(brca_clinical), len(non_brca_clinical)))

There are 1196 brca clinical info and 9224 non-brca clinical info


#### From OS to Survival

In [9]:
# Create 39 equally spaced intervals for follow-up time, from 0 to 5 years:
breaks = np.arange(0.,365.*5,365./8)
n_intervals = len(breaks)-1
timegap = breaks[1:] - breaks[:-1]

In [10]:
import nnet_survival
non_brca_y = nnet_survival.make_surv_array(non_brca_clinical['OS.time'],
                             non_brca_clinical['OS'],
                             breaks)
brca_y = nnet_survival.make_surv_array(brca_clinical['OS.time'],
                             brca_clinical['OS'],
                             breaks)

Using TensorFlow backend.


# TRAINING MODELS

In [11]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, MaxPool2D, Activation, Dropout, Flatten
from keras import regularizers, optimizers
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
import warnings
from keras.regularizers import l1
from keras.optimizers import Adam
import tensorflow as tf

In [24]:
# Using C-index as evaluation metric (using a custom callback)

from lifelines.utils import concordance_index
from keras.callbacks import Callback

class CIndex(Callback):
    """
    Callback that computes the C-index metric both on training and test data after each epoch.
    
    Arguments:
        train_x: Numpy array containing the training dataset.
        train_time: Numpy array with survival time for training dataset.
        train_event: Numpy array with survival event for training dataset.
        val_x: Numpy array containing the test dataset.
        val_time: Numpy array with survival time for test dataset.
        val_event: Numpy array with survival event for test dataset.
        filepath: path to save the model.
        years: number of years to compute c-index
        patience: patience for early-stop training
    """
    
    def __init__(self, train_x, train_time, train_event, val_x, val_time, val_event, filepath, years=2, patience=5):
        super(Callback, self).__init__()
        self.X_tr = train_x
        self.time_tr = train_time
        self.event_tr = train_event
        self.X_val = val_x
        self.time_val = val_time
        self.event_val = val_event
        self.years = years
        self.patience = patience
        self.best = 0.0
        self.wait = 0  #counter for patience
        self.best_rounds = 1
        self.counter = 0
        self.filepath = filepath
        
    def on_train_begin(self, logs={}):
        self.c_index_dict = {'c-index': [], 'val_c-index': []}

    def on_epoch_end(self, epoch, logs={}):
        self.counter +=1

        y_tr=self.model.predict_proba(self.X_tr,verbose=0)
        yr_surv=np.cumprod(y_tr[:,0:np.nonzero(breaks>365*self.years)[0][0]], axis=1)[:,-1]
        c_index_tr = concordance_index(self.time_tr, yr_surv, self.event_tr)
        y_val=self.model.predict_proba(self.X_val,verbose=0)
        yr_surv_val=np.cumprod(y_val[:,0:np.nonzero(breaks>365*self.years)[0][0]], axis=1)[:,-1]
        c_index_val = concordance_index(self.time_val, yr_surv_val, self.event_val)

        #print('\tc-index: %s - val_c-index: %s' % (str(round(c_index_tr, 4)), str(round(c_index_val, 4))))
        
        if (c_index_val > self.best):
            self.c_index_dict['c-index'].append(c_index_tr)
            self.c_index_dict['val_c-index'].append(c_index_val)

            self.best = c_index_val
            self.best_rounds = self.counter
            self.wait = 0
            self.model.save_weights(self.filepath, overwrite=True)
        else:
            if self.wait >= self.patience: #no more patience, retrieve best model
                self.model.stop_training = True
                print('Best number of rounds: %d \nValidation C-Index: %f \n' % (self.best_rounds, self.best))
                self.model.load_weights(self.filepath)
            self.wait += 1

## NON-BRCA

### Spliting data for training and testing

In [ ]:
# NON-BRCA
# Split dataset intor train and test
from sklearn.model_selection import train_test_split

X_non_brca_train, X_non_brca_test, y_non_brca_train, y_non_brca_test, train_index, test_index  = train_test_split(X_non_brca, non_brca_y, np.arange(len(X_non_brca)), test_size=0.2, random_state=42)
X_non_brca_train, X_non_brca_val, y_non_brca_train, y_non_brca_val, train_index, val_index = train_test_split(X_non_brca_train, y_non_brca_train, train_index, test_size=0.2, random_state=44)

In [ ]:
y_non_brca_train = np.array(y_non_brca_train, dtype=int)
y_non_brca_test = np.array(y_non_brca_test, dtype=int)
y_non_brca_val = np.array(y_non_brca_val, dtype=int)

In [ ]:
print("Dataset size:", X_non_brca_train.shape, X_non_brca_test.shape, X_non_brca_val.shape)

### Bayesian Optimization (non-brca)

In [ ]:
from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
space = {
    'conv_choice': hp.choice('conv_num_layers', 
                            [{'layers': 'two',
                             'units_2conv2': hp.choice('units_2conv2', [16, 32, 64, 128]),
                              '2kernel2': hp.choice('2kernel2', [3,5]),
                              '2pool2': 2,
                             'dropout_2conv2': hp.choice('dropout_2conv2', [0.25, 0.5, 0.75])
                            },
                            {'layers': 'three',
                             'units_3conv2': hp.choice('units_3conv2', [16, 32, 64, 128]),
                             '3kernel2': hp.choice('3kernel2', [3,5]),
                             '3pool2': 2,
                             'dropout_3conv2': hp.choice('dropout_3conv2', [0.25, 0.5, 0.75]),
                             'units_3conv3': hp.choice('units_3conv3', [16, 32, 64, 128]),
                             '3kernel3': hp.choice('kernel3', [3,5]),
                             '3pool3': 2,
                             'dropout_3conv3': hp.choice('dropout_3conv3', [0.25, 0.5, 0.75])
                            }]
                            ),
    'dense_choice': hp.choice('dense_num_layers', 
                            [{'layers': 'two',
                             'units_2dense2': hp.choice('units_2dense2', [16, 32, 64, 128]),
                             'dropout_2dense2': hp.choice('dropout_2dense2', [0.25, 0.5, 0.75])
                            },
                            {'layers': 'three',
                             'units_3dense2': hp.choice('units_3dense2', [16, 32, 64, 128]),
                             'dropout_3dense2': hp.choice('dropout_3dense2', [0.25, 0.5, 0.75]),
                             'units_3dense3': hp.choice('units_3dense3', [16, 32, 64, 128]),
                             'dropout_3dense3': hp.choice('dropout_3dense3', [0.25, 0.5, 0.75])
                            }]
                            ),
    'units_conv1': hp.choice('units_conv1', [16, 32, 64, 128]),
    'kernel1': hp.choice('kernel1', [3,5]),
    'pool1': 2,
    'dropout_conv1': hp.choice('dropout_conv1', [0.25, 0.5, 0.75]),
    'units_dense1': hp.choice('units_dense1', [16, 32, 64, 128]),
    'dropout_dense1': hp.choice('dropout_dense1', [0.25, 0.5, 0.75]),
    'batch_size' : hp.choice('batch_size', [16, 32, 64, 128]),
    'nb_epochs' : 100,
    'lr': hp.loguniform('lr', np.log(0.0001), np.log(0.01)),
    'activation': 'relu'                                            
}

In [ ]:
print(X_non_brca_train.shape)
print(non_brca_clinical.iloc[train_index]['OS.time'].shape)

In [ ]:
def train_non_brca(params):
    print('Params testing: ', params)
    print('\n')
    
    # Prevent GPU memory allocation problems
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    K.set_session(tf.Session(config=config))

    model = Sequential()

    # Conv layer 1 ---
    model.add(Conv2D(filters=params['units_conv1'], 
                     kernel_size=(params['kernel1'], params['kernel1']),
                     input_shape = (X_non_brca_train.shape[1],
                                    X_non_brca_train.shape[2],
                                    X_non_brca_train.shape[3])))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(MaxPool2D(pool_size=(params['pool1'], params['pool1'])))
    model.add(Dropout(params['dropout_conv1']))

    if params['conv_choice']['layers'] == 'two':
        # Conv layer 2 ---
        model.add(Conv2D(filters=params['conv_choice']['units_2conv2'], 
                     kernel_size=(params['conv_choice']['2kernel2'], params['conv_choice']['2kernel2'])))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool2D(pool_size=(params['conv_choice']['2pool2'], params['conv_choice']['2pool2'])))
        model.add(Dropout(params['conv_choice']['dropout_2conv2']))
    elif params['conv_choice']['layers'] == 'three':
        # Conv layer 2 ---
        model.add(Conv2D(filters=params['conv_choice']['units_3conv2'], 
                     kernel_size=(params['conv_choice']['3kernel2'], params['conv_choice']['3kernel2'])))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool2D(pool_size=(params['conv_choice']['3pool2'], params['conv_choice']['3pool2'])))
        model.add(Dropout(params['conv_choice']['dropout_3conv2']))
        # Conv layer 3 ---
        model.add(Conv2D(filters=params['conv_choice']['units_3conv3'], 
                     kernel_size=(params['conv_choice']['3kernel3'], params['conv_choice']['3kernel3'])))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool2D(pool_size=(params['conv_choice']['3pool3'], params['conv_choice']['3pool3'])))
        model.add(Dropout(params['conv_choice']['dropout_3conv3']))

    # Flatten layer ---
    model.add(Flatten())
    
    # Dense layer 1 ---
    model.add(Dense(params['units_dense1']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout_dense1']))

    if params['dense_choice']['layers']== 'two':
        # Dense layer 2 ---
        model.add(Dense(params['dense_choice']['units_2dense2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_2dense2']))
    elif params['dense_choice']['layers']== 'three':
        # Dense layer 2 ---
        model.add(Dense(params['dense_choice']['units_3dense2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_3dense2']))
        # Dense layer 3 ---
        model.add(Dense(params['dense_choice']['units_3dense3']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_3dense3']))
        
    # Output layer ---
    model.add(Dense(n_intervals, activation='sigmoid'))

    optimizer = Adam(lr=params['lr'])

    model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizer)

    hist_c_index = CIndex(X_non_brca_train,
                        non_brca_clinical.iloc[train_index]['OS.time'],
                        non_brca_clinical.iloc[train_index]['OS'],
                        X_non_brca_val,
                        non_brca_clinical.iloc[val_index]['OS.time'],
                        non_brca_clinical.iloc[val_index]['OS'],
                        filepath = 'keras-models/common_best_non-brca.h5')
    
    callbacks = [hist_c_index]   

    batch = params['batch_size']
    n_epoch = params['nb_epochs']

    start = timer()
    model.fit(X_non_brca_train, 
              y_non_brca_train, 
              batch_size=batch, 
              epochs=n_epoch, 
              validation_data = [X_non_brca_val, y_non_brca_val],
              callbacks=callbacks,
              verbose=0)
    run_time = timer() - start
    
    predictions = model.predict_proba(X_non_brca_test,verbose=0)
    twoyr_surv=np.cumprod(predictions[:,0:np.nonzero(breaks>365*2)[0][0]], axis=1)[:,-1]
    score = concordance_index(non_brca_clinical.iloc[test_index]['OS.time'], twoyr_surv, non_brca_clinical.iloc[test_index]['OS'])
    
    return {'loss': score*(-1),
            'params': params,
            'train_time': run_time,
            'status': STATUS_OK, 
            'rounds':hist_c_index.best_rounds}

In [ ]:
from timeit import default_timer as timer
from multiprocessing import Pool
from hyperopt import STATUS_OK

def objective_non_brca(params):
    from sklearn.metrics import roc_auc_score
    
    from keras.models import Sequential
    from keras.layers import Input, Dense, Conv1D, MaxPool1D, Activation, Dropout, Flatten
    from keras import regularizers, optimizers
    from keras.layers.normalization import BatchNormalization
    from keras.callbacks import EarlyStopping

    from keras.regularizers import l1
    from keras.optimizers import Adam
   
    with Pool(1) as p:
            trial = p.apply(train_non_brca, args=(params, ))

    return trial

In [ ]:
%%time

trials_non_brca = Trials()

best_non_brca = fmin(objective_non_brca, space, algo=tpe.suggest, trials=trials_non_brca, max_evals=50)

print (best_non_brca)
print (trials_non_brca.best_trial)

import pickle

with open("keras-models/common_hyper_opt_non-brca.pkl", 'wb') as f:
    pickle.dump([trials_non_brca, best_non_brca], f)

In [ ]:
import pickle

with open("keras-models/common_hyper_opt_non-brca.pkl", 'rb') as f:
    [trials_non_brca, best_non_brca] = pickle.load(f)

In [ ]:
trials_non_brca.best_trial

In [ ]:
del X_non_brca, X_non_brca_test, X_non_brca_train, X_non_brca_val, non_brca_clinical, non_brca_y

## BRCA

### Spliting data for training and testing

In [13]:
# BRCA
# Split dataset intor train and test
from sklearn.model_selection import train_test_split

X_brca_train, X_brca_test, y_brca_train, y_brca_test, train_index, test_index  = train_test_split(X_brca, brca_y, np.arange(len(X_brca)), test_size=0.2, random_state=42)
X_brca_train, X_brca_val, y_brca_train, y_brca_val, train_index, val_index = train_test_split(X_brca_train, y_brca_train, train_index, test_size=0.2, random_state=44)

In [14]:
y_brca_train = np.array(y_brca_train, dtype=int)
y_brca_test = np.array(y_brca_test, dtype=int)
y_brca_val = np.array(y_brca_val, dtype=int)

In [15]:
print("Dataset size:", X_brca_train.shape, X_brca_test.shape, X_brca_val.shape)

Dataset size: (764, 39, 39, 33) (240, 39, 39, 33) (192, 39, 39, 33)


### Bayesian Optimization (brca)

In [ ]:
from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
space = {
    'conv_choice': hp.choice('conv_num_layers', 
                            [{'layers': 'two',
                             'units_2conv2': hp.choice('units_2conv2', [16, 32, 64, 128]),
                              '2kernel2': hp.choice('2kernel2', [3,5]),
                              '2pool2': 2,
                             'dropout_2conv2': hp.choice('dropout_2conv2', [0.25, 0.5, 0.75])
                            },
                            {'layers': 'three',
                             'units_3conv2': hp.choice('units_3conv2', [16, 32, 64, 128]),
                             '3kernel2': hp.choice('3kernel2', [3,5]),
                             '3pool2': 2,
                             'dropout_3conv2': hp.choice('dropout_3conv2', [0.25, 0.5, 0.75]),
                             'units_3conv3': hp.choice('units_3conv3', [16, 32, 64, 128]),
                             '3kernel3': hp.choice('kernel3', [3,5]),
                             '3pool3': 2,
                             'dropout_3conv3': hp.choice('dropout_3conv3', [0.25, 0.5, 0.75])
                            }]
                            ),
    'dense_choice': hp.choice('dense_num_layers', 
                            [{'layers': 'two',
                             'units_2dense2': hp.choice('units_2dense2', [16, 32, 64, 128]),
                             'dropout_2dense2': hp.choice('dropout_2dense2', [0.25, 0.5, 0.75])
                            },
                            {'layers': 'three',
                             'units_3dense2': hp.choice('units_3dense2', [16, 32, 64, 128]),
                             'dropout_3dense2': hp.choice('dropout_3dense2', [0.25, 0.5, 0.75]),
                             'units_3dense3': hp.choice('units_3dense3', [16, 32, 64, 128]),
                             'dropout_3dense3': hp.choice('dropout_3dense3', [0.25, 0.5, 0.75])
                            }]
                            ),
    'units_conv1': hp.choice('units_conv1', [16, 32, 64, 128]),
    'kernel1': hp.choice('kernel1', [3,5]),
    'pool1': 2,
    'dropout_conv1': hp.choice('dropout_conv1', [0.25, 0.5, 0.75]),
    'units_dense1': hp.choice('units_dense1', [16, 32, 64, 128]),
    'dropout_dense1': hp.choice('dropout_dense1', [0.25, 0.5, 0.75]),
    'batch_size' : hp.choice('batch_size', [16, 32, 64, 128]),
    'nb_epochs' : 100,
    'lr': hp.loguniform('lr', np.log(0.0001), np.log(0.01)),
    'activation': 'relu'                                            
}

In [ ]:
def train_brca(params):
    print('Params testing: ', params)
    print('\n')
    
    # Prevent GPU memory allocation problems
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    K.set_session(tf.Session(config=config))

    model = Sequential()

    # Conv layer 1 ---
    model.add(Conv2D(filters=params['units_conv1'], 
                     kernel_size=(params['kernel1'], params['kernel1']),
                     input_shape = (X_brca_train.shape[1],
                                    X_brca_train.shape[2],
                                    X_brca_train.shape[3])))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(MaxPool2D(pool_size=(params['pool1'], params['pool1'])))
    model.add(Dropout(params['dropout_conv1']))

    if params['conv_choice']['layers'] == 'two':
        # Conv layer 2 ---
        model.add(Conv2D(filters=params['conv_choice']['units_2conv2'], 
                     kernel_size=(params['conv_choice']['2kernel2'], params['conv_choice']['2kernel2'])))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool2D(pool_size=(params['conv_choice']['2pool2'], params['conv_choice']['2pool2'])))
        model.add(Dropout(params['conv_choice']['dropout_2conv2']))
    elif params['conv_choice']['layers'] == 'three':
        # Conv layer 2 ---
        model.add(Conv2D(filters=params['conv_choice']['units_3conv2'], 
                     kernel_size=(params['conv_choice']['3kernel2'], params['conv_choice']['3kernel2'])))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool2D(pool_size=(params['conv_choice']['3pool2'], params['conv_choice']['3pool2'])))
        model.add(Dropout(params['conv_choice']['dropout_3conv2']))
        # Conv layer 3 ---
        model.add(Conv2D(filters=params['conv_choice']['units_3conv3'], 
                     kernel_size=(params['conv_choice']['3kernel3'], params['conv_choice']['3kernel3'])))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool2D(pool_size=(params['conv_choice']['3pool3'], params['conv_choice']['3pool3'])))
        model.add(Dropout(params['conv_choice']['dropout_3conv3']))

    # Flatten layer ---
    model.add(Flatten())
    
    # Dense layer 1 ---
    model.add(Dense(params['units_dense1']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout_dense1']))

    if params['dense_choice']['layers']== 'two':
        # Dense layer 2 ---
        model.add(Dense(params['dense_choice']['units_2dense2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_2dense2']))
    elif params['dense_choice']['layers']== 'three':
        # Dense layer 2 ---
        model.add(Dense(params['dense_choice']['units_3dense2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_3dense2']))
        # Dense layer 3 ---
        model.add(Dense(params['dense_choice']['units_3dense3']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_3dense3']))
        
    # Output layer ---
    model.add(Dense(n_intervals, activation='sigmoid'))

    optimizer = Adam(lr=params['lr'])

    model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizer)

    hist_c_index = CIndex(X_brca_train,
                        brca_clinical.iloc[train_index]['OS.time'],
                        brca_clinical.iloc[train_index]['OS'],
                        X_brca_val,
                        brca_clinical.iloc[val_index]['OS.time'],
                        brca_clinical.iloc[val_index]['OS'],
                        filepath = 'keras-models/common_best_brca.h5')
    
    callbacks = [hist_c_index]   

    batch = params['batch_size']
    n_epoch = params['nb_epochs']

    start = timer()
    model.fit(X_brca_train, 
              y_brca_train, 
              batch_size=batch, 
              epochs=n_epoch, 
              validation_data = [X_brca_val, y_brca_val],
              callbacks=callbacks,
              verbose=0)
    run_time = timer() - start
    
    predictions = model.predict_proba(X_brca_test,verbose=0)
    twoyr_surv=np.cumprod(predictions[:,0:np.nonzero(breaks>365*2)[0][0]], axis=1)[:,-1]
    score = concordance_index(brca_clinical.iloc[test_index]['OS.time'], twoyr_surv, brca_clinical.iloc[test_index]['OS'])
    
    return {'loss': score*(-1),
            'params': params,
            'train_time': run_time,
            'status': STATUS_OK, 
            'rounds':hist_c_index.best_rounds}

In [ ]:
from timeit import default_timer as timer
from multiprocessing import Pool
from hyperopt import STATUS_OK

def objective_brca(params):
    from sklearn.metrics import roc_auc_score
    
    from keras.models import Sequential
    from keras.layers import Input, Dense, Conv1D, MaxPool1D, Activation, Dropout, Flatten
    from keras import regularizers, optimizers
    from keras.layers.normalization import BatchNormalization
    from keras.callbacks import EarlyStopping

    from keras.regularizers import l1
    from keras.optimizers import Adam
   
    with Pool(1) as p:
            trial = p.apply(train_brca, args=(params, ))

    return trial

In [ ]:
%%time

trials_brca = Trials()

best_brca = fmin(objective_brca, space, algo=tpe.suggest, trials=trials_brca, max_evals=50)

print (best_brca)
print (trials_brca.best_trial)

import pickle

with open("keras-models/common_hyper_opt_brca.pkl", 'wb') as f:
    pickle.dump([trials_brca, best_brca], f)

In [ ]:
import pickle

with open("keras-models/common_hyper_opt_brca.pkl", 'rb') as f:
    [trials_brca, best_brca] = pickle.load(f)

In [ ]:
trials_brca.best_trial

## BRCA (fine tuning) 

### Train non-brca model with all non-brca data

In [13]:
print("Input data shape: {}".format(X_non_brca.shape))

Input data shape: (9224, 39, 39, 33)


In [14]:
print("Output data shape: {}".format(non_brca_y.shape))

Output data shape: (9224, 78)


In [15]:
import pickle

with open("keras-models/common_hyper_opt_non-brca.pkl", 'rb') as f:
    [trials_non_brca, best_non_brca] = pickle.load(f)

In [16]:
params = trials_non_brca.best_trial['result']['params']

In [19]:
from timeit import default_timer as timer

# Prevent GPU memory allocation problems
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.4
K.set_session(tf.Session(config=config))

model = Sequential()

# Conv layer 1 ---
model.add(Conv2D(filters=params['units_conv1'], 
                 kernel_size=(params['kernel1'], params['kernel1']),
                 input_shape = (X_non_brca.shape[1],
                                X_non_brca.shape[2],
                                X_non_brca.shape[3])))
model.add(BatchNormalization())
model.add(Activation(params['activation']))
model.add(MaxPool2D(pool_size=(params['pool1'], params['pool1'])))
model.add(Dropout(params['dropout_conv1']))

if params['conv_choice']['layers'] == 'two':
    # Conv layer 2 ---
    model.add(Conv2D(filters=params['conv_choice']['units_2conv2'], 
                 kernel_size=(params['conv_choice']['2kernel2'], params['conv_choice']['2kernel2'])))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(MaxPool2D(pool_size=(params['conv_choice']['2pool2'], params['conv_choice']['2pool2'])))
    model.add(Dropout(params['conv_choice']['dropout_2conv2']))
elif params['conv_choice']['layers'] == 'three':
    # Conv layer 2 ---
    model.add(Conv2D(filters=params['conv_choice']['units_3conv2'], 
                 kernel_size=(params['conv_choice']['3kernel2'], params['conv_choice']['3kernel2'])))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(MaxPool2D(pool_size=(params['conv_choice']['3pool2'], params['conv_choice']['3pool2'])))
    model.add(Dropout(params['conv_choice']['dropout_3conv2']))
    # Conv layer 3 ---
    model.add(Conv2D(filters=params['conv_choice']['units_3conv3'], 
                 kernel_size=(params['conv_choice']['3kernel3'], params['conv_choice']['3kernel3'])))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(MaxPool2D(pool_size=(params['conv_choice']['3pool3'], params['conv_choice']['3pool3'])))
    model.add(Dropout(params['conv_choice']['dropout_3conv3']))

# Flatten layer ---
model.add(Flatten())

# Dense layer 1 ---
model.add(Dense(params['units_dense1']))
model.add(BatchNormalization())
model.add(Activation(params['activation']))
model.add(Dropout(params['dropout_dense1']))

if params['dense_choice']['layers']== 'two':
    # Dense layer 2 ---
    model.add(Dense(params['dense_choice']['units_2dense2']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dense_choice']['dropout_2dense2']))
elif params['dense_choice']['layers']== 'three':
    # Dense layer 2 ---
    model.add(Dense(params['dense_choice']['units_3dense2']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dense_choice']['dropout_3dense2']))
    # Dense layer 3 ---
    model.add(Dense(params['dense_choice']['units_3dense3']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dense_choice']['dropout_3dense3']))

# Output layer ---
model.add(Dense(n_intervals, activation='sigmoid'))

optimizer = Adam(lr=params['lr'])

model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizer)

batch = params['batch_size']
n_epoch = params['nb_epochs'] 

## Fit model using all data
model.fit(X_non_brca, 
          non_brca_y, 
          batch_size=batch, 
          epochs=n_epoch)

Epoch 1/100
9224/9224 [==============================] - 10s 1ms/step - loss: 2.8263
Epoch 2/100
9224/9224 [==============================] - 8s 913us/step - loss: 1.6401
Epoch 3/100
9224/9224 [==============================] - 8s 919us/step - loss: 1.6030
Epoch 4/100
9224/9224 [==============================] - 8s 918us/step - loss: 1.5727
Epoch 5/100
9224/9224 [==============================] - 8s 915us/step - loss: 1.5381
Epoch 6/100
9224/9224 [==============================] - 8s 916us/step - loss: 1.5332
Epoch 7/100
9224/9224 [==============================] - 9s 924us/step - loss: 1.5125
Epoch 8/100
9224/9224 [==============================] - 8s 916us/step - loss: 1.5002
Epoch 9/100
9224/9224 [==============================] - 9s 922us/step - loss: 1.4922
Epoch 10/100
9224/9224 [==============================] - 8s 910us/step - loss: 1.4847
Epoch 11/100
9224/9224 [==============================] - 8s 916us/step - loss: 1.4735
Epoch 12/100
9224/9224 [=============================

In [21]:
predictions = model.predict_proba(X_non_brca,verbose=0)
twoyr_surv=np.cumprod(predictions[:,0:np.nonzero(breaks>365*2)[0][0]], axis=1)[:,-1]
score = concordance_index(non_brca_clinical['OS.time'], twoyr_surv, non_brca_clinical['OS'])
print(score)

0.8322895409908285


In [22]:
predictions.shape

(9224, 39)

In [23]:
model.save('keras-models/MODEL_nonBRCA_all_data.h5')

### Fine tuning with bayesian optimization

In [16]:
from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials

In [17]:
space_fine_tuning = {
    'choice': hp.choice('num_layers', 
                            [{'layers': 'one'},
                            {'layers': 'two',
                             'units_2dense1': hp.choice('units_2dense1', [16, 32, 64, 128, 256]),
                             'dropout_2dense1': hp.choice('dropout_2dense1', [0.25, 0.5, 0.75])
                            },
                            {'layers': 'three',
                             'units_3dense1': hp.choice('units_3dense1', [16, 32, 64, 128, 256]),
                             'dropout_3dense1': hp.choice('dropout_3dense1', [0.25, 0.5, 0.75]),
                             'units_3dense2': hp.choice('units_3dense2', [16, 32, 64, 128, 256]),
                             'dropout_3dense2': hp.choice('dropout_3dense2', [0.25, 0.5, 0.75])
                            }]
                            ),
    'units_output': n_intervals,
    'batch_size' : hp.choice('batch_size', [16, 32, 64, 128]),
    'nb_epochs' : 100,
    'lr': hp.loguniform('lr', np.log(0.0001), np.log(0.01)),
    'activation_hidden': 'relu',
    'activation_output': 'sigmoid'
}

In [15]:
import pickle

with open("keras-models/common_hyper_opt_non-brca.pkl", 'rb') as f:
    [trials_non_brca, best_non_brca] = pickle.load(f)

In [18]:
import nnet_survival

In [29]:
def train_fine_tuning(params):
    print('Params testing: ', params)
    print('\n')

    # Prevent GPU memory allocation problems
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    K.set_session(tf.Session(config=config))

    from keras.models import load_model

    non_brca_model = load_model('keras-models/MODEL_nonBRCA_all_data.h5', custom_objects={'loss': nnet_survival.surv_likelihood(n_intervals)})
    
    import pickle

    with open("keras-models/common_hyper_opt_non-brca.pkl", 'rb') as f:
        [trials_non_brca, best_non_brca] = pickle.load(f)
        
    if trials_non_brca.best_trial['result']['params']['dense_choice']['layers'] == 'two':
        pop_layers = 6
    elif trials_non_brca.best_trial['result']['params']['dense_choice']['layers'] == 'three':
        pop_layers = 9
    
    for i in range(pop_layers):
        non_brca_model.pop()    
    
    for layer in non_brca_model.layers:
         layer.trainable = False
    
    model = Sequential()
    
    model.add(non_brca_model)

    if params['choice']['layers']== 'two':
        # Dense layer 1 ---
        model.add(Dense(params['choice']['units_2dense1']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation_hidden']))
        model.add(Dropout(params['choice']['dropout_2dense1']))
    elif params['choice']['layers']== 'three':
        # Dense layer 1 ---
        model.add(Dense(params['choice']['units_3dense1']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation_hidden']))
        model.add(Dropout(params['choice']['dropout_3dense1']))
        # Dense layer 2 ---
        model.add(Dense(params['choice']['units_3dense2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation_hidden']))
        model.add(Dropout(params['choice']['dropout_3dense2']))
        
    # Output layer ---
    model.add(Dense(params['units_output'], activation=params['activation_output']))

    optimizer = Adam(lr=params['lr'])
    
    model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizer)
    
    #model.summary()
    
    batch = params['batch_size']
    n_epoch = params['nb_epochs']

    hist_c_index = CIndex(X_brca_train,
                    brca_clinical.iloc[train_index]['OS.time'],
                    brca_clinical.iloc[train_index]['OS'],
                    X_brca_val,
                    brca_clinical.iloc[val_index]['OS.time'],
                    brca_clinical.iloc[val_index]['OS'],
                    filepath = 'keras-models/common_best_fine-tuning_sigmoid.h5')

    callbacks = [hist_c_index]    
    
    ## Fit model using training data
    start = timer()
    model.fit(X_brca_train, 
              y_brca_train, 
              batch_size=batch, 
              epochs=n_epoch, 
              validation_data = [X_brca_val, y_brca_val],
              callbacks=callbacks,
              verbose=0)
    run_time = timer() - start
    
    predictions = model.predict_proba(X_brca_test,verbose=0)
    twoyr_surv=np.cumprod(predictions[:,0:np.nonzero(breaks>365*2)[0][0]], axis=1)[:,-1]
    score = concordance_index(brca_clinical.iloc[test_index]['OS.time'], twoyr_surv, brca_clinical.iloc[test_index]['OS'])

    # Return loss
    return {'loss': score*(-1),
            'params': params,
            'train_time': run_time,
            'status': STATUS_OK, 
            'rounds':hist_c_index.best_rounds}

In [20]:
from timeit import default_timer as timer
from multiprocessing import Pool
from hyperopt import STATUS_OK

def objective_fine_tuning(params):
    from sklearn.metrics import roc_auc_score
    
    from keras.models import Sequential
    from keras.layers import Input, Dense, Conv2D, MaxPool2D, Activation, Dropout, Flatten
    from keras import regularizers, optimizers
    from keras.layers.normalization import BatchNormalization
    from keras.callbacks import EarlyStopping

    from keras.regularizers import l1
    from keras.optimizers import Adam
       
    with Pool(1) as p:
            trial = p.apply(train_fine_tuning, args=(params, ))

    return trial

In [30]:
trials_fine_tuning = Trials()

best_fine_tuning = fmin(fn=objective_fine_tuning, space=space_fine_tuning, algo=tpe.suggest, trials=trials_fine_tuning, max_evals=100)

print (best_fine_tuning)
print (trials_fine_tuning.best_trial)

Params testing:                                      
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 16, 'choice': {'dropout_3dense1': 0.25, 'dropout_3dense2': 0.5, 'layers': 'three', 'units_3dense1': 256, 'units_3dense2': 16}, 'lr': 0.00029398123589792995, 'nb_epochs': 100, 'units_output': 39}
Best number of rounds: 1                             
Validation C-Index: 0.429187 

Params testing:                                                                
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 16, 'choice': {'dropout_2dense1': 0.75, 'layers': 'two', 'units_2dense1': 256}, 'lr': 0.0029833847860556563, 'nb_epochs': 100, 'units_output': 39}
Best number of rounds: 7                                                       
Validation C-Index: 0.595694 

Params testing:                                                                
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 64, 'choice': {'drop

Best number of rounds: 1                                                        
Validation C-Index: 0.496651 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 32, 'choice': {'dropout_2dense1': 0.25, 'layers': 'two', 'units_2dense1': 64}, 'lr': 0.00010386924278179812, 'nb_epochs': 100, 'units_output': 39}
Best number of rounds: 1                                                        
Validation C-Index: 0.384211 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 32, 'choice': {'dropout_2dense1': 0.25, 'layers': 'two', 'units_2dense1': 64}, 'lr': 0.00010001004422514721, 'nb_epochs': 100, 'units_output': 39}
Best number of rounds: 1                                                        
Validation C-Index: 0.384211 

Params testing:                                         

Best number of rounds: 1                                                        
Validation C-Index: 0.546411 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 32, 'choice': {'dropout_2dense1': 0.75, 'layers': 'two', 'units_2dense1': 64}, 'lr': 0.00018731747654837297, 'nb_epochs': 100, 'units_output': 39}
Best number of rounds: 11                                                       
Validation C-Index: 0.439713 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 64, 'choice': {'layers': 'one'}, 'lr': 0.0001919289379915116, 'nb_epochs': 100, 'units_output': 39}
Best number of rounds: 27                                                       
Validation C-Index: 0.622488 

Params testing:                                                                 
{'activation_hidden': 

Best number of rounds: 12                                                       
Validation C-Index: 0.655981 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 64, 'choice': {'dropout_3dense1': 0.75, 'dropout_3dense2': 0.5, 'layers': 'three', 'units_3dense1': 32, 'units_3dense2': 128}, 'lr': 0.00016252748897000065, 'nb_epochs': 100, 'units_output': 39}
Best number of rounds: 5                                                        
Validation C-Index: 0.632536 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 16, 'choice': {'dropout_3dense1': 0.75, 'dropout_3dense2': 0.5, 'layers': 'three', 'units_3dense1': 32, 'units_3dense2': 128}, 'lr': 0.0011620409325767225, 'nb_epochs': 100, 'units_output': 39}
Best number of rounds: 1                                                 

Best number of rounds: 1                                                        
Validation C-Index: 0.357416 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 32, 'choice': {'dropout_3dense1': 0.75, 'dropout_3dense2': 0.5, 'layers': 'three', 'units_3dense1': 16, 'units_3dense2': 16}, 'lr': 0.0013967610896370927, 'nb_epochs': 100, 'units_output': 39}
Best number of rounds: 2                                                        
Validation C-Index: 0.561722 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 32, 'choice': {'layers': 'one'}, 'lr': 0.00038137154679976945, 'nb_epochs': 100, 'units_output': 39}
Best number of rounds: 7                                                        
Validation C-Index: 0.621053 

Params testing:                                         

{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 64, 'choice': {'layers': 'one'}, 'lr': 0.0013604337569418697, 'nb_epochs': 100, 'units_output': 39}
Best number of rounds: 3                                                        
Validation C-Index: 0.620096 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 64, 'choice': {'dropout_3dense1': 0.25, 'dropout_3dense2': 0.75, 'layers': 'three', 'units_3dense1': 64, 'units_3dense2': 128}, 'lr': 0.0019361457517919642, 'nb_epochs': 100, 'units_output': 39}
Best number of rounds: 1                                                        
Validation C-Index: 0.613876 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 64, 'choice': {'dropout_3dense1': 0.25, 'dropout_3dense2': 0.75, 'layers': 'three', 'units_

In [31]:
with open("keras-models/common_hyper_opt_fine-tuning_sigmoid.pkl", 'wb') as f:
    pickle.dump([trials_fine_tuning, best_fine_tuning], f)